<font size="6"><b>Making Decisions with Trees</b></font>

<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Introduction-" data-toc-modified-id="Introduction--1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Introduction <a id="intro"></a></a></span></li><li><span><a href="#Architecture-" data-toc-modified-id="Architecture--2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Architecture <a id="arc"></a></a></span><ul class="toc-item"><li><ul class="toc-item"><li><span><a href="#Root-Node:-" data-toc-modified-id="Root-Node:--2.0.1"><span class="toc-item-num">2.0.1&nbsp;&nbsp;</span>Root Node: <a id="root"></a></a></span></li><li><span><a href="#Leaf-Node" data-toc-modified-id="Leaf-Node-2.0.2"><span class="toc-item-num">2.0.2&nbsp;&nbsp;</span>Leaf Node<a id="leaf"></a></a></span></li><li><span><a href="#Decision-Node" data-toc-modified-id="Decision-Node-2.0.3"><span class="toc-item-num">2.0.3&nbsp;&nbsp;</span>Decision Node<a id="dec"></a></a></span></li></ul></li></ul></li><li><span><a href="#Data" data-toc-modified-id="Data-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Data<a id="data"></a></a></span><ul class="toc-item"><li><span><a href="#Splitting-Data-on-Features" data-toc-modified-id="Splitting-Data-on-Features-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>Splitting Data on Features<a id="splitfeat"></a></a></span></li></ul></li><li><span><a href="#Split-Criteria-" data-toc-modified-id="Split-Criteria--4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Split Criteria <a id="split"></a></a></span><ul class="toc-item"><li><span><a href="#Gini-Measure" data-toc-modified-id="Gini-Measure-4.1"><span class="toc-item-num">4.1&nbsp;&nbsp;</span>Gini Measure<a id="gini"></a></a></span></li></ul></li><li><span><a href="#Subsequent-Splits-" data-toc-modified-id="Subsequent-Splits--5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Subsequent Splits <a id="moresplit"></a></a></span></li><li><span><a href="#Decision-Tree-Code:-" data-toc-modified-id="Decision-Tree-Code:--6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Decision Tree Code: <a id="tree"></a></a></span></li></ul></div>

## Introduction <a id="intro"></a>

Decision trees are a simple yet powerful tool in predictive modeling. 

Unlike other predictive models, such as neural networks, we are able to decipher the inner working of a decision trees. In other words, they are easier to interpret. We can trace back the decisions that led to a particular prediction.

The tree starts at the root, sequentially making comparisons until a decision is made. Think of it as a nested if statement. The decision node is where the prediction of a class is made. 

In this reading, we will spend our time on classification trees and a simple implementaton of the code.

## Architecture <a id="arc"></a>

![](architecture.png)

#### Root Node: <a id="root"></a>
The first best feature to split the data. The notion of "best" will be determined by the splitting criteria used in the decision tree algorithm.

#### Leaf Node<a id="leaf"></a>

#### Decision Node<a id="dec"></a>

## Data<a id="data"></a>
Lets begin by looking at the population data set:

In [483]:
# Create Sample of Data
import pandas as pd
import numpy as np
Emotion=['sick','sick','sick','notsick','notsick','sick','notsick','notsick']
Temperature = ['under','over','under','under','over','over','under','over']
StayHome=['N','Y','Y','N','Y','N','N','Y']
df=pd.DataFrame (list(zip(Emotion,Temperature,StayHome)),
                columns=["Emotion","Temperature","StayHome"])
print(df.sort_values(['Temperature','StayHome']).reset_index(drop=True))

   Emotion Temperature StayHome
0     sick        over        N
1     sick        over        Y
2  notsick        over        Y
3  notsick        over        Y
4     sick       under        N
5  notsick       under        N
6  notsick       under        N
7     sick       under        Y


We make decisions in our lives everyday. Although we make decisions so often, we may not stop to think exactly how we made them.

Let's think about a decision many of us have come across: **Going to Work**. Without realizing it, we tend to ask ourselves sequential questions, that lead us to our decision.

Imagine that the data above captures 8 individuals and their actions. We would like to create conditional statements in which to split the data in order to make predictions on unobserved decisions to go to work.

At the root node, we will split the data set based on the unique values of a given feature. For example, if we used the 'Emotion' feature we would split the data on 'sick' and 'not sick'

In [449]:
df[df.iloc[:,0]=='sick'].sort_values('StayHome')

,Emotion,Temperature,StayHome
0,sick,under,N
5,sick,over,N
1,sick,over,Y
2,sick,under,Y


In [444]:
df[df.iloc[:,0]=='notsick'].sort_values('StayHome')

,Emotion,Temperature,StayHome
3,notsick,under,N
6,notsick,under,N
4,notsick,over,Y
7,notsick,over,Y


If we used the feature 'Temperature' we wouls split it on 'under' and 'over'.

### Splitting Data on Features<a id="splitfeat"></a>
Lets start to code a split of the data based on each unique value of a feature. We will first need the index position of the features, beacuse we will not always have the outcome variable in the last column.

In [450]:
#Identify the Outcome Variable
Outcome='StayHome'
#Creates a list of index without the outcome column
outcome=df.columns.get_loc(Outcome)
col_ls=list(range(len(df.columns)))
col_ls.remove(outcome)
#print feature positions
col_ls

[0, 1]

Next we will create a copy of the dataset and gather the unique values in a feature:

In [491]:
 #create copy of data and get unique values in each feature
new_df=df
names,count=np.unique(new_df.iloc[:,col_ls[0]],return_counts=True)
# returns how many unique feature there are in this split
splitdf=len(names)
for i in range(0,len(names)):
    print("We have split on feature:","(",new_df.columns[0],")","where it equals","(",names[i],")","and has",count[i],"observations \n")

We have split on feature: ( Emotion ) where it equals ( notsick ) and has 4 observations 

We have split on feature: ( Emotion ) where it equals ( sick ) and has 4 observations 



Using the code above we have not yet split the data frame. We only have the unique values of the feature and the split count (which will come in handy later on). In order to split the data we will use the index **col_ls** and the unique values **names**.

In [452]:
for n in names:
    print("-----------------------------------------")
    print(new_df[new_df.iloc[:,col_ls[0]]==n],"\n","\n")

-----------------------------------------
   Emotion Temperature StayHome
3  notsick       under        N
4  notsick        over        Y
6  notsick       under        N
7  notsick        over        Y 
 

-----------------------------------------
  Emotion Temperature StayHome
0    sick       under        N
1    sick        over        Y
2    sick       under        Y
5    sick        over        N 
 



## Split Criteria <a id="split"></a>

But how do we choose which feature to split on first?

There are two methods we have previously discussed to decide on the best split: [Gini Measure](https://sites.google.com/view/stevenloaiza/machine-learning/decision-trees/gini-impurity-index?authuser=0) and [Information Gain via entropy](https://sites.google.com/view/stevenloaiza/machine-learning/decision-trees/information-gain?authuser=0). We will use the former for the remainder of the discussion.

### Gini Measure<a id="gini"></a>

Recall our definition:

Def: Gini Impurity tells us what is the probability of misclassifying an observation, using the distribution from the split, from a random sample conditional on our split.

Mathematical definition:

$Ginx = \sum_{n}^{i=1} p_i \cdot \sum_{i \ne j} \cdot p_i = \sum_{n}^{i=1} p_i \cdot (1-p_i)$

To generate the gini impurity measure for the entire split we will need to weight them accordingly.

To obtain the impurity measure of each split you will need to pass the split data set through the code. It will then evauate the impurity of each **Leaf Node**. Below is an example for one split set, but this would need to be done for all possible splits and then weighted.

In [482]:
splitdata=new_df[new_df.iloc[:,col_ls[0]]==names[1]]
# outcome is defined above

#gets the unique outcome values
value=np.unique(df.iloc[:,outcome])
#This will be used later when we computed the weighted gini value
denS=len(splitdata)
impurity=0
                 
for values in value:
            p=splitdata.iloc[:,outcome].eq(values).sum()/denS
            impurity +=p*(1-p)
print ("The gini impurity index is: ",impurity,"for \n \n", splitdata.sort_values("StayHome").reset_index(drop=True))

The gini impurity index is:  0.5 for 
 
   Emotion Temperature StayHome
0    sick       under        N
1    sick        over        N
2    sick        over        Y
3    sick       under        Y


Use the formula above to confirm that the impurity at this split is indeed (0.5). The formula to evaluate is $Gini= (1/2)*(1-1/2)+ (1/2)*(1-1/2)$.

Note that the split on "not sick" is exactly the same. 

To get the weighted gini impurity measure we will need to weight each split by its proportion to the data before the split. For instance before the split there were 8 observations, then when we split on "sick" and "not sick", 4 went bto each side respectively.

Hence the weighted gini measure is Weighted_Gini= $(4/8)*(0.5)+ (4/8)*(0.5)$.

In [495]:
#create empty list to hold each splits impurity and split count
gini_metric=list()
den_split=list()
        
for n in names:
    splitdata=new_df[new_df.iloc[:,col_ls[0]]==n]
    value=np.unique(df.iloc[:,outcome])
#This will be used later when we computed the weighted gini value
    denS=len(splitdata)
    impurity=0
                 
    for values in value:
            p=splitdata.iloc[:,outcome].eq(values).sum()/denS
            impurity +=p*(1-p)
    gini_metric.append(impurity)
    den_split.append(denS)


feature=splitdf
# get the proporation of the split for current feature
den_split1=[x/sum(den_split[0:feature]) for x in den_split[0:feature]]
weighted_gini=sum(np.asarray(den_split1)*np.asarray(gini_metric[0:feature]))

print("The weighted gini impurtiy measure is:",weighted_gini)

The weighted gini impurtiy measure is: 0.5


We will combine all the code above to iterate over each split of a feature. The empty list "w_gini" is keeping track of the weighted gini, then it will return the name of the feature with the lowest weighted gini.

In [506]:
# Generalize our gini Function
def Gini(splitdata,outcome):
    # Get unique Outcome values
    value=np.unique(df.iloc[:,outcome])
    denS=len(splitdata)
    impurity=0
    for values in value:
                    p=splitdata.iloc[:,outcome].eq(values).sum()/denS
                    impurity +=p*(1-p)
    return (impurity,denS)

#Tree
def SimpleTree(df,Outcome):
    if type(Outcome)==str:
        
         #empty list to hold information
        splitdf=list()
        gini_metric=list()
        w_gini=list()
        den_split=list()
        split_data=list()
        P=list()
        
        #Creates a list of index without the outcome column
        outcome=df.columns.get_loc(Outcome)
        col_ls=list(range(len(df.columns)))
        col_ls.remove(outcome)
       
        #begin loops
        for col in col_ls:
            #create copy of data and get unique values in each feature
            new_df=df
            names,count=np.unique(new_df.iloc[:,col],return_counts=True)
            #Returns the number of splits per feature 
            splitdf.append(len(names))
            
            for n in names:
                #returns the columns such that it equals the split criteria
                split_df=new_df[new_df.iloc[:,col]==n]
                #add df to list
                
                split_data.append(split_df)
                
        # Get unique Outcome values
        value=np.unique(df.iloc[:,outcome])
        
        for data in split_data:
            #calcualte Gini at each split
            impurity,denS=Gini(data,outcome)
            #append lists
            gini_metric.append(impurity)
            den_split.append(denS)
        # For loop which will iterate on the number of splits per feature to get the WEIGHTED GINI METRIC
        for feature in splitdf:
            # get the proporation of the split for current feature
            den_split1=[x/sum(den_split[0:feature]) for x in den_split[0:feature]]
            weighted_gini=sum(np.asarray(den_split1)*np.asarray(gini_metric[0:feature]))
            #remove the gini and feature count split for the next iteration
                              
            w_gini.append(weighted_gini)
            del(gini_metric[0:feature], den_split[0:feature])
    else: print("Place Outcome Variable Name in \'Single\' quotes")
    #return feature with the smallest impurity index
    f_col=w_gini.index(min(w_gini))
    #returns the name
    bestsplit=df.columns[f_col]
    return(col_ls,w_gini,bestsplit,split_data)
#------------------------------------------------------------------------------------------


In [505]:
col_ls,w_gini,bestsplit,k=SimpleTree(df,"StayHome")
for col in col_ls:
           print("Feature","(",df.columns[col],")","Weighted Gini Impurity Index is:", w_gini[col],"\n")
print("The Best split is with feature","(",bestsplit,")")

Feature ( Emotion ) Weighted Gini Impurity Index is: 0.5 

Feature ( Temperature ) Weighted Gini Impurity Index is: 0.375 

The Best split is with feature ( Temperature )


## Subsequent Splits <a id="moresplit"></a>

Up to this point we have successful decided how to initiate the split from the root node. Our next challenge is to use these data frame after the splits, and perfrom the same tasks. In other words, once the split occurs on temperature, we do not want to split on temperature again instead the algorithm will use different feature(s) and choose the optimal split.

## Decision Tree Code: <a id="tree"></a>

Lets put it all together now.